In [1]:
import os, sys
from glob import glob
import pandas as pd
import requests
from tqdm import tqdm
import shutil

In [2]:
dfs = []
for path in glob('*/output/*.csv'):
    if 'val' in path:
        continue # Skip Validation Set
    df = pd.read_csv(path)
    df['source'] = path.split('/')[-1][:-4]
    dfs.append(df.loc[df['match_score'] >= 0.555,:])
df = pd.concat(dfs).sort_values(['source', 'match_score']).reset_index(drop=True)
df.to_csv('filtered_image_meta.csv', index=False)

In [6]:
paths = []
for idx, (img_url, source) in tqdm(enumerate(zip(df['img_url'], df['source']))):
    file_path = f"IMG-SCRAP_{idx}_{source}_{img_url.split('?')[0].split('/')[-1][-200:]}"
    if '.' not in file_path[-7:]: # If there is no extension, add .jpg extension
        file_path = f'{file_path}.jpg'
    out_path = f'./gathered_images/{file_path}'

    if not os.path.exists(out_path):
        try:
            response = requests.get(img_url, stream=True, timeout=20, verify=True)
            with open(out_path, 'wb') as out_file:
                shutil.copyfileobj(response.raw, out_file)
            paths.append(out_path)
        except Exception as e: 
            print(f'failed downloading  {img_url} ...')
            print(e)
            paths.append(None)
    else:
        paths.append(out_path)

499it [00:00, 2390.81it/s]

failed downloading  http://jeffenjane.com/wp-content/uploads/2013/12/DSC05960-1024x680.jpg ...
HTTPConnectionPool(host='jeffenjane.com', port=80): Max retries exceeded with url: /wp-content/uploads/2013/12/DSC05960-1024x680.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f2769a4bac0>: Failed to establish a new connection: [Errno -5] No address associated with hostname'))
failed downloading  https://baotangphunu.org.vn/wp-content/uploads/2014/04/2-1.jpg ...
('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


2136it [00:00, 3588.03it/s]

failed downloading  https://canossacollegespc.com.ph/wp-content/uploads/2018/10/42211151_10214724157151485_2865786893549699072_n-960x350.jpg ...
('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
failed downloading  https://anlacgreensymphony.tinbds24h.com.vn/wp-content/uploads/sites/12/2020/09/hinh-anh-du-an-anlac-green-symphony.jpg ...
('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
failed downloading  https://namhuongcorp.com.vn/wp-content/uploads/2019/10/CW3A7541-1.jpg ...
('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
failed downloading  https://vanlanguni.edu.vn/images/nhom-nhap-tin/2017/11/14/dong-phuong-hoc/asia-travel-photography-2.jpg ...
HTTPSConnectionPool(host='vanlanguni.edu.vn', port=443): Max retries exceeded with url: /images/nhom-nhap-tin/2017/11/14/dong-phuong-hoc/asia-travel-photography-2.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f2769

2541it [00:01, 1429.20it/s]

...

failed downloading  https://solvillas.vn/wp-content/uploads/2020/06/80d4a2c10e7af424ad6b-530x313.jpg ...
('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


3318it [00:23, 54.83it/s] 

failed downloading  http://cdn1.vietnamtourism.org.vn/images/content/2012-04-26.09.01.22-4.jpg ...
HTTPConnectionPool(host='cdn1.vietnamtourism.org.vn', port=80): Read timed out. (read timeout=20)
failed downloading  http://chinatownhotel.co.th/wp-content/uploads/2017/10/chinatown_hotel_receptionist-2.jpg ...
HTTPConnectionPool(host='chinatownhotel.co.th', port=80): Max retries exceeded with url: /wp-content/uploads/2017/10/chinatown_hotel_receptionist-2.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f27ad358820>: Failed to establish a new connection: [Errno -2] Name or service not known'))
failed downloading  https://alliancerealestate.com.vn/wp-content/uploads/2019/12/0d57da190af4f3aaaae5-554x360-c-center.jpg ...
('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))


3786it [00:43, 86.35it/s]

failed downloading  http://cdn1.vietnamtourism.org.vn/images/content/2012-04-26.08.57.05-1.jpg ...
HTTPConnectionPool(host='cdn1.vietnamtourism.org.vn', port=80): Read timed out. (read timeout=20)


In [12]:
df['local_path'] = paths
df.loc[~df['local_path'].isna(),:]
df.to_csv('filtered_image_meta.csv', index=False)